# Kapitola 15: Programovani s omezenimi - Problem osmi dam

**Blok 2: Prohledavaci algoritmy a optimalizace**

---

## Co se naucite

V teto kapitole:
- Pochopite **Constraint Satisfaction Programming (CSP)** - zcela novy zpusob reseni problemu
- Vyresime slavny **problem osmi dam** na sachovnici
- Naučite se pouzivat **Google OR-Tools** - profesionalni CSP solver
- Pochopite rozdil mezi **imperativnim** a **deklarativnim** programovanim
- Aplikujete CSP na dalsi problemy: **Sudoku**, **rozvrzeni hodin**

---

## Motivace: Jiny zpusob mysleni

Doposud jsme psali algoritmy, ktere AI rikaly **JAK** resit problem:
- "Projdi vsechny sousedy, pridej je do fronty..."
- "Spocitej heuristiku, vyjmi uzel s nejnizsi prioritou..."

Dnes se naucime zcela jiny pristup: receme AI **CO** chceme, ne jak to ma udelat.
Definujeme **pravidla a omezeni**, a necháme specializovany solver, aby nasel reseni.

In [ ]:
# Instalace Google OR-Tools
!pip install ortools -q

# Dalsi knihovny
import numpy as np
import matplotlib.pyplot as plt
from ortools.sat.python import cp_model
import time

print("OR-Tools uspesne nainstalovany!")
print("Pripraveno pro Constraint Satisfaction Programming.")

---

## Cast 1: Problem osmi dam

### Pravidla hry

**Ukol**: Umistete 8 sachovych dam na standardni sachovnici 8x8 tak, aby se zadne dve
navzajem neohrozovaly.

Dama v sachu ohrozuje:
- Cely svuj **radek** (horizontalne)
- Cely svuj **sloupec** (vertikalne)
- Obe sve **diagonaly**

### Proc je to tezke?

- Celkem existuje C(64, 8) = **4,426,165,368** zpusobu jak umistit 8 dam na 64 poli
- Ale jen **92** z nich je spravnych!
- Zkouset vsechny moznosti by trvalo velmi dlouho

In [ ]:
def zobraz_sachovnici(queens, title="Sachovnice s damami"):
    """
    Zobrazi sachovnici s umistenim dam.
    
    Args:
        queens: Seznam pozic dam (queens[sloupec] = radek)
        title: Titulek obrazku
    """
    n = len(queens)
    
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Vytvorime sachovnicovy vzor
    board = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if (i + j) % 2 == 0:
                board[i, j] = 1
    
    ax.imshow(board, cmap='binary', alpha=0.3)
    
    # Pridame damy
    for sloupec, radek in enumerate(queens):
        ax.text(sloupec, radek, '\u265B', fontsize=36, 
                ha='center', va='center', color='darkred')
    
    # Mrizka
    for i in range(n + 1):
        ax.axhline(i - 0.5, color='black', linewidth=1)
        ax.axvline(i - 0.5, color='black', linewidth=1)
    
    ax.set_xlim(-0.5, n - 0.5)
    ax.set_ylim(n - 0.5, -0.5)
    ax.set_xticks(range(n))
    ax.set_yticks(range(n))
    ax.set_xticklabels(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'][:n])
    ax.set_yticklabels(range(1, n + 1))
    ax.set_title(title, fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

# Priklad jednoho reseni (rucne zadane)
priklad_reseni = [0, 4, 7, 5, 2, 6, 1, 3]
zobraz_sachovnici(priklad_reseni, "Priklad jednoho reseni problemu 8 dam")

---

## Cast 2: Mysleni v podminkach (Constraint Thinking)

### Modelovani problemu

Misto ptani se "kam mam polozit damu?" se ptame:
**"Jake podminky musi splnovat platne reseni?"**

### Nase promenne

V kazdem sloupci musi byt prave jedna dama. Proto:
- Vytvorime 8 promennych: `q[0], q[1], ..., q[7]`
- Kazda promenna `q[c]` rika, na kterem **radku** je dama ve **sloupci c**
- Hodnoty: 0 az 7

### Nase omezeni (Constraints)

1. **Sloupce**: Vyreseno automaticky - kazdy sloupec ma jednu promennou
2. **Radky**: Vsechny hodnoty musi byt ruzne: `AllDifferent(q[0], q[1], ..., q[7])`
3. **Diagonaly**: Pro kazdy par dam (i, j): `|q[i] - q[j]| != |i - j|`

In [ ]:
def over_reseni(queens):
    """
    Overi, zda je dane reseni platne.
    
    Returns:
        True pokud je reseni platne, jinak False s popisem chyby
    """
    n = len(queens)
    
    # Kontrola radku - vsechny hodnoty musi byt ruzne
    if len(set(queens)) != n:
        return False, "Dve damy na stejnem radku!"
    
    # Kontrola diagonal
    for i in range(n):
        for j in range(i + 1, n):
            # Pokud |radek_i - radek_j| == |sloupec_i - sloupec_j|, jsou na diagonale
            if abs(queens[i] - queens[j]) == abs(i - j):
                return False, f"Damy na sloupcich {i} a {j} jsou na stejne diagonale!"
    
    return True, "Reseni je platne!"


# Testujeme nas priklad
platne, zprava = over_reseni(priklad_reseni)
print(f"Reseni {priklad_reseni}")
print(f"Vysledek: {zprava}")

# Testujeme spatne reseni
spatne_reseni = [0, 1, 2, 3, 4, 5, 6, 7]  # Vsechny na diagonale!
platne, zprava = over_reseni(spatne_reseni)
print(f"\nReseni {spatne_reseni}")
print(f"Vysledek: {zprava}")

---

## Cast 3: Reseni s Google OR-Tools

OR-Tools je profesionalni knihovna od Googlu pro optimalizaci a CSP.
Obsahuje CP-SAT solver - jeden z nejrychlejsich solveru na svete.

### Kroky reseni

1. **Vytvorime model** - kontejner pro promenne a omezeni
2. **Definujeme promenne** - queens[0..7] s hodnotami 0..7
3. **Pridame omezeni** - pravidla, ktera musi platit
4. **Spustime solver** - necháme AI najit reseni

In [ ]:
def resi_n_dam_ortools(n=8, najdi_vsechna=False, max_reseni=None):
    """
    Resi problem N dam pomoci OR-Tools CP-SAT solveru.
    
    Args:
        n: Velikost sachovnice a pocet dam
        najdi_vsechna: Pokud True, hleda vsechna reseni
        max_reseni: Maximalni pocet reseni k nalezeni
    
    Returns:
        Seznam nalezenych reseni
    """
    # 1. Vytvoreni modelu
    model = cp_model.CpModel()
    
    # 2. Definice promennych
    # queens[c] = r znamena, ze ve sloupci c je dama na radku r
    queens = [model.NewIntVar(0, n - 1, f'queen_{i}') for i in range(n)]
    
    # 3. Definice omezeni
    
    # 3a. Vsechny damy musi byt na ruznych radcich
    model.AddAllDifferent(queens)
    
    # 3b. Zadne dve damy nesmi byt na stejne diagonale
    # Diagonala 1: queen[i] + i musi byt ruzne pro vsechny i
    # Diagonala 2: queen[i] - i musi byt ruzne pro vsechny i
    diag1 = [queens[i] + i for i in range(n)]
    diag2 = [queens[i] - i for i in range(n)]
    model.AddAllDifferent(diag1)
    model.AddAllDifferent(diag2)
    
    # 4. Vytvoreni solveru
    solver = cp_model.CpSolver()
    
    if najdi_vsechna:
        # Callback pro sber vsech reseni
        class SolutionCollector(cp_model.CpSolverSolutionCallback):
            def __init__(self, queens, max_solutions=None):
                cp_model.CpSolverSolutionCallback.__init__(self)
                self.queens = queens
                self.solutions = []
                self.max_solutions = max_solutions
            
            def on_solution_callback(self):
                solution = [self.Value(q) for q in self.queens]
                self.solutions.append(solution)
                
                if self.max_solutions and len(self.solutions) >= self.max_solutions:
                    self.StopSearch()
        
        collector = SolutionCollector(queens, max_reseni)
        solver.parameters.enumerate_all_solutions = True
        status = solver.Solve(model, collector)
        
        return collector.solutions
    else:
        # Najdeme jen jedno reseni
        status = solver.Solve(model)
        
        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            return [[solver.Value(q) for q in queens]]
        else:
            return []


# Najdeme jedno reseni
print("Hledam jedno reseni problemu 8 dam...")
start = time.time()
reseni = resi_n_dam_ortools(8, najdi_vsechna=False)
cas = time.time() - start

if reseni:
    print(f"Reseni nalezeno za {cas*1000:.2f} ms!")
    print(f"Pozice dam: {reseni[0]}")
    zobraz_sachovnici(reseni[0], "Reseni nalezene OR-Tools")

In [ ]:
# Najdeme VSECHNA reseni
print("Hledam vsechna reseni problemu 8 dam...")
start = time.time()
vsechna_reseni = resi_n_dam_ortools(8, najdi_vsechna=True)
cas = time.time() - start

print(f"Nalezeno {len(vsechna_reseni)} reseni za {cas*1000:.2f} ms!")

# Zobrazime prvnich 4 reseni
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

for idx, ax in enumerate(axes.flatten()):
    queens = vsechna_reseni[idx]
    n = len(queens)
    
    # Sachovnicovy vzor
    board = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if (i + j) % 2 == 0:
                board[i, j] = 1
    
    ax.imshow(board, cmap='binary', alpha=0.3)
    
    # Damy
    for sloupec, radek in enumerate(queens):
        ax.text(sloupec, radek, '\u265B', fontsize=24, 
                ha='center', va='center', color='darkred')
    
    # Mrizka
    for i in range(n + 1):
        ax.axhline(i - 0.5, color='black', linewidth=0.5)
        ax.axvline(i - 0.5, color='black', linewidth=0.5)
    
    ax.set_xlim(-0.5, n - 0.5)
    ax.set_ylim(n - 0.5, -0.5)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(f"Reseni #{idx + 1}: {queens}", fontsize=10)

plt.suptitle("Prvni 4 z 92 reseni problemu 8 dam", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---

## Cast 4: Experimenty s velikosti sachovnice

Co se stane, kdyz zmenime velikost sachovnice?

In [ ]:
def benchmark_n_dam():
    """
    Testuje rychlost reseni pro ruzne velikosti sachovnice.
    """
    velikosti = [4, 5, 6, 7, 8, 9, 10, 11, 12]
    vysledky = []
    
    print("Benchmarking problemu N dam")
    print("="*50)
    print(f"{'N':<5} {'Pocet reseni':<15} {'Cas (ms)':<15}")
    print("-"*50)
    
    for n in velikosti:
        start = time.time()
        reseni = resi_n_dam_ortools(n, najdi_vsechna=True)
        cas = (time.time() - start) * 1000
        
        vysledky.append({
            'n': n,
            'pocet': len(reseni),
            'cas_ms': cas
        })
        print(f"{n:<5} {len(reseni):<15} {cas:<15.2f}")
    
    print("="*50)
    return vysledky

vysledky = benchmark_n_dam()

# Vizualizace
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ns = [v['n'] for v in vysledky]
pocty = [v['pocet'] for v in vysledky]
casy = [v['cas_ms'] for v in vysledky]

# Graf 1: Pocet reseni
ax1 = axes[0]
ax1.bar(ns, pocty, color='steelblue')
ax1.set_xlabel('Velikost sachovnice (N)')
ax1.set_ylabel('Pocet reseni')
ax1.set_title('Pocet reseni problemu N dam')
ax1.set_xticks(ns)
for i, (n, p) in enumerate(zip(ns, pocty)):
    ax1.text(n, p + max(pocty)*0.02, str(p), ha='center', fontsize=9)

# Graf 2: Cas reseni
ax2 = axes[1]
ax2.plot(ns, casy, 'o-', color='green', linewidth=2, markersize=8)
ax2.set_xlabel('Velikost sachovnice (N)')
ax2.set_ylabel('Cas reseni (ms)')
ax2.set_title('Cas hledani vsech reseni')
ax2.set_xticks(ns)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Cast 5: Dalsi CSP problemy

CSP neni jen pro damy! Pouziva se pro:
- **Sudoku** - cisla 1-9 v kazdem radku, sloupci, ctverci
- **Rozvrh hodin** - ucitele, ucebny, predmety bez kolizi
- **Planovani smen** - zakonné pauzy, kvalifikace
- **Logistika** - nakladani kamionu, trasy

### Priklad: Reseni Sudoku

In [ ]:
def resi_sudoku(puzzle):
    """
    Resi Sudoku pomoci CSP.
    
    Args:
        puzzle: 9x9 pole, 0 znamena prazdne policko
    
    Returns:
        Vyresene Sudoku nebo None
    """
    model = cp_model.CpModel()
    
    # Vytvorime promenne pro kazde policko
    cells = {}
    for i in range(9):
        for j in range(9):
            if puzzle[i][j] != 0:
                # Policko je uz vyplnene
                cells[(i, j)] = model.NewIntVar(puzzle[i][j], puzzle[i][j], f'cell_{i}_{j}')
            else:
                # Prazdne policko - hodnoty 1-9
                cells[(i, j)] = model.NewIntVar(1, 9, f'cell_{i}_{j}')
    
    # Omezeni pro radky
    for i in range(9):
        model.AddAllDifferent([cells[(i, j)] for j in range(9)])
    
    # Omezeni pro sloupce
    for j in range(9):
        model.AddAllDifferent([cells[(i, j)] for i in range(9)])
    
    # Omezeni pro 3x3 ctverec
    for box_i in range(3):
        for box_j in range(3):
            box_cells = []
            for i in range(3):
                for j in range(3):
                    box_cells.append(cells[(box_i * 3 + i, box_j * 3 + j)])
            model.AddAllDifferent(box_cells)
    
    # Vyresime
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        result = [[solver.Value(cells[(i, j)]) for j in range(9)] for i in range(9)]
        return result
    return None


def zobraz_sudoku(puzzle, title="Sudoku"):
    """Zobrazi Sudoku."""
    fig, ax = plt.subplots(figsize=(8, 8))
    
    for i in range(9):
        for j in range(9):
            val = puzzle[i][j]
            if val != 0:
                ax.text(j + 0.5, 8.5 - i, str(val), ha='center', va='center', fontsize=20)
    
    # Mrizka
    for i in range(10):
        lw = 2 if i % 3 == 0 else 0.5
        ax.axhline(i, color='black', linewidth=lw)
        ax.axvline(i, color='black', linewidth=lw)
    
    ax.set_xlim(0, 9)
    ax.set_ylim(0, 9)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_aspect('equal')
    plt.show()


# Priklad Sudoku
sudoku_zadani = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

print("Zadani Sudoku:")
zobraz_sudoku(sudoku_zadani, "Sudoku - Zadani")

print("\nResim Sudoku...")
start = time.time()
reseni = resi_sudoku(sudoku_zadani)
cas = time.time() - start

if reseni:
    print(f"Vyreseno za {cas*1000:.2f} ms!")
    zobraz_sudoku(reseni, "Sudoku - Reseni")
else:
    print("Zadani nema reseni!")

---

## Cast 6: Porovnani pristupu

### Imperativni vs Deklarativni programovani

| Aspekt | Imperativni (BFS, A*) | Deklarativni (CSP) |
|--------|----------------------|--------------------|
| Co piseme | JAK hledat reseni | CO chceme nalezt |
| Slozitost kodu | Vyssi | Nizsi |
| Flexibilita | Manualni optimalizace | Automaticka optimalizace |
| Vhodne pro | Specificke problemy | Problemy s omezenimi |

In [ ]:
# Porovnani: Backtracking vs CSP pro problem dam

def backtracking_n_dam(n):
    """
    Resi problem N dam klasickym backtrackingem (imperativni pristup).
    """
    reseni = []
    
    def je_bezpecne(queens, radek, sloupec):
        for c, r in enumerate(queens):
            if r == radek:
                return False
            if abs(r - radek) == abs(c - sloupec):
                return False
        return True
    
    def backtrack(queens, sloupec):
        if sloupec == n:
            reseni.append(list(queens))
            return
        
        for radek in range(n):
            if je_bezpecne(queens, radek, sloupec):
                queens.append(radek)
                backtrack(queens, sloupec + 1)
                queens.pop()
    
    backtrack([], 0)
    return reseni


# Porovnani rychlosti
print("Porovnani: Backtracking vs OR-Tools CSP")
print("="*60)
print(f"{'N':<5} {'Backtracking (ms)':<20} {'OR-Tools (ms)':<20} {'Zrychleni':<15}")
print("-"*60)

for n in [8, 10, 12]:
    # Backtracking
    start = time.time()
    res_bt = backtracking_n_dam(n)
    cas_bt = (time.time() - start) * 1000
    
    # OR-Tools
    start = time.time()
    res_ot = resi_n_dam_ortools(n, najdi_vsechna=True)
    cas_ot = (time.time() - start) * 1000
    
    zrychleni = cas_bt / cas_ot if cas_ot > 0 else float('inf')
    
    print(f"{n:<5} {cas_bt:<20.2f} {cas_ot:<20.2f} {zrychleni:<15.1f}x")
    
    # Overeni, ze oba nasli stejny pocet reseni
    assert len(res_bt) == len(res_ot), f"Rozdilny pocet reseni pro N={n}!"

print("="*60)
print("\nOR-Tools je optimalizovany solver s pokrocilymi technikami!")

---

## Cviceni k procviceni

### Cviceni 1: Vetsi sachovnice
Zkuste vyresit problem pro N=14 a N=16 dam. Kolik reseni existuje?

### Cviceni 2: Jiny hlavolam
Naprogramujte CSP resic pro "Magic Square" (magicky ctverec 3x3).

### Cviceni 3: Graf-barveni
Obarvete mapu Ceske republiky (kraje) tak, aby zadne dva sousedni kraje nemely stejnou barvu.

### Cviceni 4: Kryptaritmetika
Vyresit: SEND + MORE = MONEY (kazde pismeno = jina cislice)

In [ ]:
# Prostor pro vase reseni - Cviceni 4: SEND + MORE = MONEY

def resi_kryptaritmetiku():
    """
    Resi problem SEND + MORE = MONEY.
    Kazde pismeno predstavuje jinou cislici 0-9.
    """
    model = cp_model.CpModel()
    
    # Promenne pro kazde pismeno
    # VAS KOD ZDE:
    # S = model.NewIntVar(1, 9, 'S')  # S nemuze byt 0 (prvni cislice)
    # ...
    
    # Omezeni: vsechna pismena musi byt ruzna
    # model.AddAllDifferent([S, E, N, D, M, O, R, Y])
    
    # Omezeni: SEND + MORE = MONEY
    # model.Add(S*1000 + E*100 + N*10 + D + M*1000 + O*100 + R*10 + E 
    #           == M*10000 + O*1000 + N*100 + E*10 + Y)
    
    # Vyresit a vratit vysledek
    pass

print("Cviceni 4: Doplnte kod vyse a vyresit SEND + MORE = MONEY")

---

## Shrnuti kapitoly

### Co jsme se naucili:

1. **Constraint Satisfaction Programming** - deklarativni pristup k reseni problemu
2. **Problem N dam** - klasicky CSP problem a jeho reseni
3. **Google OR-Tools** - profesionalni knihovna pro CSP
4. **Modelovani omezeni** - jak preformulovat problem do CSP
5. **Aplikace CSP** - Sudoku, rozvrhy, logistika

### Klicove koncepty:

| Pojem | Vyznam |
|-------|--------|
| **Promenna** | Neznama hodnota, kterou hledame |
| **Domena** | Mnozina moznych hodnot promenne |
| **Omezeni (Constraint)** | Pravidlo, ktere musi platit |
| **Solver** | Algoritmus, ktery hleda reseni |
| **AllDifferent** | Vsechny promenne musi mit ruzne hodnoty |

### Dalsi kroky:

V dalsi kapitole se podivame na **Geneticke algoritmy** - jak se AI muze "vyvijet"
k lepsim resenim inspirovanim Darwinovou evolucni teorii!

---

*Kapitola 15 - Programovani s omezenimi*